In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

sc.parallelize(range(100)).count()

In [ ]:
def joinOverTimeCondition(val1,val2):
if val1[0]<timedelta(0) and val2[0]>timedelta(0):
  return val2
elif val2[0]<timedelta(0) and val1[0]>timedelta(0):
  return val1
else:
  return val1 if val1[0]<val2[0] else val2

def merger(x):
    newDic = {}
    for key in x[0]:
      newDic[key] = x[0][key]
    for key in x[1]:
      if key in newDic:
          if x[1][key] == newDic[key]:
            continue
          newDic[key+"_2"] = x[1][key]
      else:
         newDic[key] = x[1][key]
    return newDic
      
obj1Key = ["lineId","itemId"]
obj2Key =  ["lineId","lineName"]

joinKeyListObj1 = ["lineId"]
joinKeyListObj2 =  ["lineId"]

tsKey1 = "ts"
tsKey2 = "date"

rdd1.map(lambda x: (tuple([x.get(key) for key in joinKeyListObj1]),x))\
          .join(Obj2.data.map(lambda x: (tuple([x.get(key) for key in joinKeyListObj2]),x)))\
          .map(lambda x:  (tuple([(key,x[1][0].get(key)) for key in joinKeyListObj1]+[(key,x[1][1].get(key)) 
                                                                                      for key in joinKeyListObj2]),
                           (x[1][1].get(tsKey1)-x[1][0].get(tsKey2),x))).reduceByKey(lambda x,y: joinOverTimeCondition(x,y))\
          .map(lambda x: merger(x[1][1][1]))